In [63]:
import requests
import pandas as pd
import json

#https://www.sec.gov/edgar/sec-api-documentation
#https://data.sec.gov/submissions/CIK##########.json

ticker_url = "https://www.sec.gov/include/ticker.txt"


response = requests.get(ticker_url)
df = pd.read_csv(ticker_url, header=None, sep='\s+')

#rename dataframe columns
df = df.rename(columns={0: 'ticker-name', 1: 'ticker-cik'})

#print dataframe columns
#for col in df.columns:
#    print(col)

#print all dataframe rows
#for index, row in df.iterrows():
#    print(row['ticker-name'], row['ticker-cik'])


#get first row
ticker_name = df.loc[0,'ticker-name']
ticker_cik = df.loc[0,'ticker-cik']

#add zeros to ticker_cik 
ticker_cik = (str(ticker_cik).zfill(10))
#print(ticker_cik)

#company url
#https://data.sec.gov/submissions/CIK0000320193.json
url = "https://data.sec.gov/submissions/CIK{}.json".format(ticker_cik)
print(url)


headers = {
    'User-Agent': 'svetlozar draganov svetlozardraganov@gmail.com'
}

#download json file
ticker_json = requests.get(url, headers=headers)
ticker_json = json.loads(ticker_json.text)

#print json data
print(ticker_json['tickers'])
print(ticker_json['phone'])

#ticker form type
#print(ticker_json['filings']['recent']['form'])
ticker_range = len(ticker_json['filings']['recent']['form'])

#find 10-Q annual reports
#for i in range(ticker_range):
#    if (ticker_json['filings']['recent']['form'][i]) == '10-Q':
#        accession_number = ticker_json['filings']['recent']['accessionNumber'][i]
#        accession_number = accession_number.replace('-','') #remove '-' from accession number
#        primary_document = ticker_json['filings']['recent']['primaryDocument'][i]
#        #print(i,accession_number,primary_document)
#        
#        url =  'https://www.sec.gov/ix?doc=/Archives/edgar/data/{}/{}/{}'.format(ticker_cik, accession_number, primary_document)
#        print(url)


#sample url
#https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000032019322000059/aapl-20220326.htm        
#print(ticker_json['filings']['files'])
#'https://www.sec.gov/Archives/edgar/data/0000320193/000032019322000059/'
#https://www.sec.gov/Archives/edgar/data/0000320193/



#https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python

#https://www.sec.gov/edgar/sec-api-documentation
#https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json 


#Apple company facts
url = "https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json"
company_facts = requests.get(url, headers=headers)
company_facts_json = json.loads(company_facts.text)
print(company_facts_json['cik'], company_facts_json['entityName'])
print(company_facts_json['facts']['dei']['EntityCommonStockSharesOutstanding']['label'])
print(company_facts_json['facts']['dei']['EntityCommonStockSharesOutstanding']['description'])
shares_range = len(company_facts_json['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares'])

#for i in range(shares_range):
#    print(company_facts_json['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares'][i]['form'])


# Get Company Shares Outstanding
shares_outstanding_json = company_facts_json['facts']['dei']['EntityCommonStockSharesOutstanding']['units']
df_company_facts_shares = pd.json_normalize(shares_outstanding_json, record_path =['shares'])
#print(df_company_facts_shares)

# Get Company Public Float
public_float_json = company_facts_json['facts']['dei']['EntityPublicFloat']['units']
df_company_facts_shares = pd.json_normalize(public_float_json, record_path =['USD'])
#print(df_company_facts_shares)


#get Apple Revenue
url = "https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/RevenueFromContractWithCustomerExcludingAssessedTax.json"
revenue = requests.get(url, headers=headers)
revenue_json = json.loads(revenue.text)
revenue_json = revenue_json['units']
df_revenue = pd.json_normalize(revenue_json, record_path =['USD'])
print(df_revenue)



https://data.sec.gov/submissions/CIK0000320193.json
['AAPL']
(408) 996-1010
320193 Apple Inc.
Entity Common Stock, Shares Outstanding
Indicate number of shares or other units outstanding of each of registrant's classes of capital or common stock or other ownership interests, if and as stated on cover of related periodic report. Where multiple classes or units exist define each class/interest by adding class of stock items such as Common Class A [Member], Common Class B [Member] or Partnership Interest [Member] onto the Instrument [Domain] of the Entity Listings, Instrument.
         start         end           val                  accn    fy  fp  \
0   2016-09-25  2017-09-30  229234000000  0000320193-19-000119  2019  FY   
1   2017-10-01  2017-12-30   88293000000  0000320193-19-000010  2019  Q1   
2   2017-10-01  2017-12-30   88293000000  0000320193-19-000119  2019  FY   
3   2017-10-01  2018-03-31  149430000000  0000320193-19-000066  2019  Q2   
4   2017-12-31  2018-03-31   6113700000

In [40]:
import json
json_file = """{
    "school_name": "ABC primary school",
    "class": "Year 1",
    "students": [
    {
        "id": "A001",
        "name": "Tom",
        "math": 60,
        "physics": 66,
        "chemistry": 61
    },
    {
        "id": "A002",
        "name": "James",
        "math": 89,
        "physics": 76,
        "chemistry": 51
    },
    {
        "id": "A003",
        "name": "Jenny",
        "math": 79,
        "physics": 90,
        "chemistry": 78
    }]
}"""

data = json.loads(json_file)
# Flatten data
df_nested_list = pd.json_normalize(data, record_path =['students'])


#print(df_nested_list)

     id   name  math  physics  chemistry
0  A001    Tom    60       66         61
1  A002  James    89       76         51
2  A003  Jenny    79       90         78
